In [1]:
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Téléchargement des stopwords pour le nettoyage
nltk.download('stopwords')
from nltk.corpus import stopwords

# Chargement du dataset
data = pd.read_csv("fake.csv")

# Suppression des doublons et des valeurs manquantes
data = data.drop_duplicates().dropna(subset=['text'])

[nltk_data] Downloading package stopwords to C:\Users\BOKA
[nltk_data]     ESTHER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Fonction de nettoyage du texte
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Supprimer la ponctuation
    text = " ".join([word for word in text.split() if word not in stopwords.words('english')])  # Suppression des stopwords
    return text

# Application du nettoyage
data["clean_text"] = data["text"].apply(clean_text)

In [ ]:
# Vectorisation avec TF-IDF

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data["clean_text"])
y = data["label"]

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


[nltk_data] Downloading package stopwords to C:\Users\BOKA
[nltk_data]     ESTHER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


: 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialisation et entraînement du modèle
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

# Prédictions et évaluation
y_pred = rf_model.predict(X_test)
print(f"Précision du modèle Random Forest : {accuracy_score(y_test, y_pred):.2f}")

# Sauvegarde du modèle
import joblib
joblib.dump(rf_model, "random_forest_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Définition du modèle neuronal
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')  # Sortie binaire
])

# Compilation du modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraînement
model.fit(X_train.toarray(), y_train, epochs=5, batch_size=32, validation_data=(X_test.toarray(), y_test))

# Sauvegarde du modèle
model.save("fake_news_nn_model.h5")
